## Optymalne ogrzewanie domu - analiza rozkładu ciepła i efektywności energetycznej

---



In [30]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


## Odczytanie temperatur

In [31]:
f=open('/content/drive/MyDrive/temperatury.txt','r')
temperatury=f.read()
wiersze=temperatury.split(';')
czasy=wiersze[1].split(",")
chlodno=wiersze[3].split(",")
zimno=wiersze[5].split(",")
bardzo_zimno=wiersze[7].split(",")


## Grzejniki pod oknem


In [32]:
import numpy as np
import matplotlib.pyplot as plt

class Room:
    def __init__(self, m,n, windows, doors, tops, heater1, heater2):
        self.M=m
        self.N=n
        self.windows=windows
        self.doors=doors
        self.tops=tops
        self.h1=heater1
        self.h2=heater2

        ib1=[i for i in range(m*n) if i%m==0 ]
        ib2=[i for i in range(m*n) if (i+1)%m==0]
        ib3=[i for i in range(m*n) if i<m]
        ib4=[i for i in range(m*n) if i>m*(n-1)]

        ib1prim=[i for i in range(m*n) if i%m==1 ]
        ib2prim=[i for i in range(m*n) if (i+2)%m==0]
        ib3prim=[i+m for i in range(m*n) if i<m]
        ib4prim=[i-m for i in range(m*n) if i>m*(n-1)]

        self.walls=ib1+ib2+ib3+ib4
        self.wallsprim=ib1prim+ib2prim+ib3prim+ib4prim

        self.laplasjan()
        self.x=np.linspace(0,self.M,1)
        self.y=np.linspace(0,self.N,1)
        self.X,self.Y=np.meshgrid(self.x,self.y)
        self.u=np.zeros((len(times), self.M * self.N))
        self.u[0, :] = [18+273.15]*(self.N*self.M)
        self.u[0,self.windows]=[okno]*len(self.windows)

        self.epsilon=800/(1.2*1*0.05*1005)
        self.power=0

    def laplasjan(self):
        D2x = np.diag(-2 * np.ones(self.M)) + np.diag(np.ones(self.M- 1), 1) + np.diag(np.ones(self.M - 1), -1)
        D2y = np.diag(-2 * np.ones(self.N)) + np.diag(np.ones(self.N - 1), 1) + np.diag(np.ones(self.N - 1), -1)
        self.L = np.kron(np.eye(self.N), D2x) + np.kron(D2y, np.eye(self.M))
    def update(self,t, okno, typ=1):
        self.u[t, :] = self.u[t-1, :] + (ht / hx**2) * np.matmul(self.L, self.u[t-1, :])
        if(self.u[t, int(self.M*self.N/2+ self.M/2)]>21+273.15): heater_on=0
        else: heater_on=1
        if(typ==1):
          self.u[t, self.h1] += self.epsilon * heater_on
          self.power+=self.epsilon * heater_on
        if(typ==2):
          self.u[t, self.h2] += self.epsilon * heater_on
          self.power+=self.epsilon * heater_on

        self.u[t, self.walls] = self.u[t, self.wallsprim]
        self.u[t, self.windows] = [okno]*len(self.windows)

class Apartment:
    def __init__(self, rooms):
        self.rooms = rooms

        ib1=[i for i in range(K*K) if i%K==0 ]
        ib2=[i for i in range(K*K) if (i+1)%K==0]
        ib3=[i for i in range(K*K) if i<K]
        ib4=[i for i in range(K*K) if i>=K*(K-1)]
        ib5=[i for i in range(int(2/5*K*K), int(2/5*K*K+K)) ]
        ib6=[i for i in range(0, int(2/5*K*K)) if i%K==int(3/5*K)]
        ib7=[i for i in range(int(2/5*K*K-K), int(2/5*K*K)) ]
        ib8=[i for i in range(0, int(2/5*K*K)) if (i+1)%K==int(3/5*K)]

        i=len(ib8)
        del ib8[int(i*1/4): int(i*3/4)]
        del ib6[int(i*1/4): int(i*3/4)]
        del ib5[20:40]
        del ib7[20:40]

        del ib3[20:39]
        del ib3[51:69]
        del ib4[19:39]
        del ib4[49:69]

        self.walls=ib1+ib2+ib3+ib4+ib5+ib6+ib7+ib8

        self.u=np.zeros((len(times), K*K))
        self.merge(0)

    def update(self, t, typ):
        for room in self.rooms:
            room.update(t, typ)
    def merge(self,t):
     # 1 3
     # 0 2
      pok1=self.rooms[0]
      pok2=self.rooms[1]
      pok3=self.rooms[2]

      pok1.u[t, pok1.doors], pok2.u[t, pok2.doors[0]] = (pok1.u[t, pok1.doors]+ pok2.u[t, pok2.doors[0]])/2, (pok1.u[t, pok1.doors]+ pok2.u[t, pok2.doors[0]])/2
      pok2.u[t, pok2.doors[1]], pok3.u[t, pok3.doors] = (pok2.u[t, pok2.doors[1]]+ pok3.u[t, pok3.doors])/2, (pok2.u[t, pok2.doors[1]]+ pok3.u[t, pok3.doors])/2


      self.u[t, range(pok1.tops[0], pok1.tops[3]+1)]=pok1.u[t,:]-[273.15]*6000
      p2=[i for i in range(pok2.tops[0], pok2.tops[3]+1) if i % K <= pok2.tops[2]]
      p3=[i for i in range(pok3.tops[0], pok3.tops[3]+1) if i % K >= pok3.tops[0]]
      self.u[t, p2]=pok2.u[t,:]-[273.15]*2400
      self.u[t, p3]=pok3.u[t,:]-[273.15]*1600
      self.u[t,self.walls]=[np.nan]*len(self.walls)



In [33]:
T=24 #doba ma 24 godziny
ht=0.05 #robimy krok co 3 minut
times=np.arange(ht,T,ht)
#print(times)

K=100 #rozmiar mieszkania w ilości kratek, rozpatrujemy kwadrat [0,50]x[0,50] 1 kratka =10 cm mieszkania
dim=50 #jaki przedział
hx=dim/K
#print(hx)

#POKOJ 1
okno=float(chlodno[0])+273.15
M=K
N=int(3*K/5)
D=range(int(M/5), int(2*M/5))
Wi1=range(int(N*M - 1-M/10-M/5), int(N*M - 1-M/10))
Wi2=range(int(N*M-1-4*M/5), int(N*M-1-3*M/5))
Wi = np.unique(np.concatenate([Wi1,Wi2]))
G11=range(int(N*M-1-13*M/10), int(N*M-1-11*M/10))
G12=range(int(N*M-1-9*M/5), int(N*M-1-8*M/5))
G1 = np.unique(np.concatenate([G11,G12]))
G21=[int(N*M/3+k*M+1) for k in range (1,int(N/3))]
G22=[int(N*M/3+k*M+1+(M-1)) for k in range (1,int(N/3)+M)]
G2 = np.unique(np.concatenate([G21,G22]))
Tops=(int(2*K**2/5), int(K**2-K), int(2*K**2/5+K-1), int(K**2-1)) #LB,LU,RB,RU


pokoj1=Room(M,N,Wi,D,Tops,G1,G2)

#POKOJ 2
M=int(3*K/5)
N=int(2*K/5)
D1=range(int(N*M-1-2*M/3), int(N*M-1-M/3))
D2=[int(M*N/4+k*M-1) for k in range(1,int(N/2)+1)]
D = [D1,D2]
Wi=range(int(M/3), int(2*M/3 - 1))
G1=range(int(4*M/3), int(5*M/3-1))
G2=[int(N/4*M+k*M+1) for k in range(1, int(N/2))]
Tops=(0, int(2/5*K*(K-1)-M), int(3/5*K-1), int(2/5*K*(K-1)+3/5*K-1-M))

pokoj2=Room(M,N,Wi,D,Tops,G1,G2)

# POKOJ 3'
M=int(2*K/5)
N=int(2*K/5)
D=[int(M*N/4+k*M) for k in range(0,int(N/2))]
Wi=range(int(M/4), int(3*M/4 - 1))
G1=range(int(M/4+M), int(3*M/4 - 1+M))
G2=[int((N/4+1)*M)-2+k*M for k in range(1, int(N/2))]
Tops=(int(3/5*K), int(2/5*K*(K-1)+3/5*K-M-20),K-1, int(2/5*K**2-1))

pokoj3=Room(M,N,Wi,D,Tops,G1,G2)

mieszkanie=Apartment([pokoj1,pokoj2,pokoj3])

powers=[]
powersprim=[]

In [34]:
for t in range(1,len(times)):
  okno=float(chlodno[t])+273.15
  pokoj1.update(t, okno,1)
  pokoj2.update(t, okno,1)
  pokoj3.update(t, okno,1)
  mieszkanie.merge(t)

cieplo=pokoj1.power+pokoj2.power+pokoj3.power
powers.append(cieplo)


In [35]:
# Tworzenie animacji
X,Y=np.meshgrid(np.linspace(0,50,K), np.linspace(0,50,K))
fig, ax = plt.subplots(figsize=(4, 4))
pcm = ax.pcolormesh(X, Y, mieszkanie.u[0,:].reshape(K,K), shading='auto')
colorbar = fig.colorbar(pcm, ax=ax)
ax.set_title("Ewolucja mapy ciepła")
ax.set_xlabel("x")
ax.set_ylabel("y")

# Funkcja aktualizująca ramki animacji
def update(frame):
    data = mieszkanie.u[frame,:].reshape(K,K)
    pcm.set_array(data.ravel())

    # Dynamiczna aktualizacja zakresów
    pcm.set_clim(vmin=15, vmax=25)
    colorbar.update_normal(pcm)  # Odświeżenie zakresu colorbar

    ax.set_title(f"Na dworze: chłodno, t = {frame:.2f}")
    return pcm,

step = 5
frames = range(0, len(times), step)
ani = animation.FuncAnimation(fig, update, frames=frames, blit=False)
plt.close()
plt.rc('animation', html='jshtml')
ani
ani.save("chlodno_okna.gif", writer="pillow")
from google.colab import files
files.download("chlodno_okna.gif")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
okno=float(zimno[0])+273.15
#POKOJ 1
M=K
N=int(3*K/5)
D=range(int(M/5), int(2*M/5))
Wi1=range(int(N*M - 1-M/10-M/5), int(N*M - 1-M/10))
Wi2=range(int(N*M-1-4*M/5), int(N*M-1-3*M/5))
Wi = np.unique(np.concatenate([Wi1,Wi2]))
G11=range(int(N*M-1-13*M/10), int(N*M-1-11*M/10))
G12=range(int(N*M-1-9*M/5), int(N*M-1-8*M/5))
G1 = np.unique(np.concatenate([G11,G12]))
G21=[int(N*M/3+k*M+1) for k in range (1,int(N/3))]
G22=[int(N*M/3+k*M+1+(M-1)) for k in range (1,int(N/3))]
G2 = np.unique(np.concatenate([G21,G22]))
Tops=(int(2*K**2/5), int(K**2-K), int(2*K**2/5+K-1), int(K**2-1)) #LB,LU,RB,RU


pokoj1=Room(M,N,Wi,D,Tops,G1,G2)

#POKOJ 2
M=int(3*K/5)
N=int(2*K/5)
D1=range(int(N*M-1-2*M/3), int(N*M-1-M/3))
D2=[int(M*N/4+k*M-1) for k in range(1,int(N/2)+1)]
D = [D1,D2]
Wi=range(int(M/3), int(2*M/3 - 1))
G1=range(int(4*M/3), int(5*M/3-1))
G2=[int(N/4*M+k*M+1) for k in range(1, int(N/2))]
Tops=(0, int(2/5*K*(K-1)-M), int(3/5*K-1), int(2/5*K*(K-1)+3/5*K-1-M))

pokoj2=Room(M,N,Wi,D,Tops,G1,G2)

# POKOJ 3'
M=int(2*K/5)
N=int(2*K/5)
D=[int(M*N/4+k*M) for k in range(0,int(N/2))]
Wi=range(int(M/4), int(3*M/4 - 1))
G1=range(int(M/4+M), int(3*M/4 - 1+M))
G2=[int((N/4+1)*M)-2+k*M for k in range(1, int(N/2))]
Tops=(int(3/5*K), int(2/5*K*(K-1)+3/5*K-M-20),K-1, int(2/5*K**2-1))

pokoj3=Room(M,N,Wi,D,Tops,G1,G2)

mieszkanie=Apartment([pokoj1,pokoj2,pokoj3])

for t in range(1,len(times)):
  okno=float(zimno[t])+273.15
  pokoj1.update(t, okno,1)
  pokoj2.update(t, okno,1)
  pokoj3.update(t, okno,1)
  mieszkanie.merge(t)

cieplo=pokoj1.power+pokoj2.power+pokoj3.power
powers.append(cieplo)
# Tworzenie animacji
X,Y=np.meshgrid(np.linspace(0,50,K), np.linspace(0,50,K))
fig, ax = plt.subplots(figsize=(4, 4))
pcm = ax.pcolormesh(X, Y, mieszkanie.u[0,:].reshape(K,K), shading='auto')
colorbar = fig.colorbar(pcm, ax=ax)
ax.set_title("Ewolucja mapy ciepła")
ax.set_xlabel("x")
ax.set_ylabel("y")

# Funkcja aktualizująca ramki animacji
def update(frame):
    data = mieszkanie.u[frame,:].reshape(K,K)
    pcm.set_array(data.ravel())

    # Dynamiczna aktualizacja zakresów
    pcm.set_clim(vmin=15, vmax=25)
    colorbar.update_normal(pcm)  # Odświeżenie zakresu colorbar

    ax.set_title(f"Na dworze: zimno, t = {frame:.2f}")
    return pcm,

step = 5
frames = range(0, len(times), step)
ani = animation.FuncAnimation(fig, update, frames=frames, blit=False)
plt.close()
plt.rc('animation', html='jshtml')
ani
ani.save("zimno_okna.gif", writer="pillow")
from google.colab import files
files.download("zimno_okna.gif")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
okno=float(bardzo_zimno[0])+273.15
#POKOJ 1
M=K
N=int(3*K/5)
D=range(int(M/5), int(2*M/5))
Wi1=range(int(N*M - 1-M/10-M/5), int(N*M - 1-M/10))
Wi2=range(int(N*M-1-4*M/5), int(N*M-1-3*M/5))
Wi = np.unique(np.concatenate([Wi1,Wi2]))
G11=range(int(N*M-1-13*M/10), int(N*M-1-11*M/10))
G12=range(int(N*M-1-9*M/5), int(N*M-1-8*M/5))
G1 = np.unique(np.concatenate([G11,G12]))
G21=[int(N*M/3+k*M+1) for k in range (1,int(N/3))]
G22=[int(N*M/3+k*M+1+(M-1)) for k in range (1,int(N/3))]
G2 = np.unique(np.concatenate([G21,G22]))
Tops=(int(2*K**2/5), int(K**2-K), int(2*K**2/5+K-1), int(K**2-1)) #LB,LU,RB,RU


pokoj1=Room(M,N,Wi,D,Tops,G1,G2)

#POKOJ 2
M=int(3*K/5)
N=int(2*K/5)
D1=range(int(N*M-1-2*M/3), int(N*M-1-M/3))
D2=[int(M*N/4+k*M-1) for k in range(1,int(N/2)+1)]
D = [D1,D2]
Wi=range(int(M/3), int(2*M/3 - 1))
G1=range(int(4*M/3), int(5*M/3-1))
G2=[int(N/4*M+k*M+1) for k in range(1, int(N/2))]
Tops=(0, int(2/5*K*(K-1)-M), int(3/5*K-1), int(2/5*K*(K-1)+3/5*K-1-M))

pokoj2=Room(M,N,Wi,D,Tops,G1,G2)

# POKOJ 3'
M=int(2*K/5)
N=int(2*K/5)
D=[int(M*N/4+k*M) for k in range(0,int(N/2))]
Wi=range(int(M/4), int(3*M/4 - 1))
G1=range(int(M/4+M), int(3*M/4 - 1+M))
G2=[int((N/4+1)*M)-2+k*M for k in range(1, int(N/2))]
Tops=(int(3/5*K), int(2/5*K*(K-1)+3/5*K-M-20),K-1, int(2/5*K**2-1))

pokoj3=Room(M,N,Wi,D,Tops,G1,G2)

mieszkanie=Apartment([pokoj1,pokoj2,pokoj3])

for t in range(1,len(times)):
  okno=float(bardzo_zimno[t])+273.15
  pokoj1.update(t, okno,1)
  pokoj2.update(t, okno,1)
  pokoj3.update(t, okno,1)
  mieszkanie.merge(t)

cieplo=pokoj1.power+pokoj2.power+pokoj3.power
powers.append(cieplo)
# Tworzenie animacji
X,Y=np.meshgrid(np.linspace(0,50,K), np.linspace(0,50,K))
fig, ax = plt.subplots(figsize=(4, 4))
pcm = ax.pcolormesh(X, Y, mieszkanie.u[0,:].reshape(K,K), shading='auto')
colorbar = fig.colorbar(pcm, ax=ax)
ax.set_title("Ewolucja mapy ciepła")
ax.set_xlabel("x")
ax.set_ylabel("y")

# Funkcja aktualizująca ramki animacji
def update(frame):
    data = mieszkanie.u[frame,:].reshape(K,K)
    pcm.set_array(data.ravel())

    # Dynamiczna aktualizacja zakresów
    pcm.set_clim(vmin=15, vmax=25)
    colorbar.update_normal(pcm)  # Odświeżenie zakresu colorbar

    ax.set_title(f"Na dworze: bardzo zimno, t = {frame:.2f}")
    return pcm,

step = 5
frames = range(0, len(times), step)
ani = animation.FuncAnimation(fig, update, frames=frames, blit=False)
plt.close()
plt.rc('animation', html='jshtml')
ani
ani.save("bardzo_zimno_okna.gif", writer="pillow")
from google.colab import files
files.download("bardzo_zimno_okna.gif")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Grzejniki nie pod oknem

In [38]:
okno=float(chlodno[0])+273.15
#POKOJ 1
M=K
N=int(3*K/5)
D=range(int(M/5), int(2*M/5))
Wi1=range(int(N*M - 1-M/10-M/5), int(N*M - 1-M/10))
Wi2=range(int(N*M-1-4*M/5), int(N*M-1-3*M/5))
Wi = np.unique(np.concatenate([Wi1,Wi2]))
G11=range(int(N*M-1-13*M/10), int(N*M-1-11*M/10))
G12=range(int(N*M-1-9*M/5), int(N*M-1-8*M/5))
G1 = np.unique(np.concatenate([G11,G12]))
G21=[int(N*M/3+k*M+1) for k in range (1,int(N/3))]
G22=[int(N*M/3+k*M+1+(M-1)-2) for k in range (1,int(N/3))]
G2 = np.unique(np.concatenate([G21,G22]))
print(G2)
Tops=(int(2*K**2/5), int(K**2-K), int(2*K**2/5+K-1), int(K**2-1)) #LB,LU,RB,RU



pokoj1=Room(M,N,Wi,D,Tops,G1,G2)

#POKOJ 2
M=int(3*K/5)
N=int(2*K/5)
D1=range(int(N*M-1-2*M/3), int(N*M-1-M/3))
D2=[int(M*N/4+k*M-1) for k in range(1,int(N/2)+1)]
D = [D1,D2]
Wi=range(int(M/3), int(2*M/3 - 1))
G1=range(int(4*M/3), int(5*M/3-1))
G2=[int(N/4*M+k*M+1) for k in range(1, int(N/2))]
Tops=(0, int(2/5*K*(K-1)-M), int(3/5*K-1), int(2/5*K*(K-1)+3/5*K-1-M))

pokoj2=Room(M,N,Wi,D,Tops,G1,G2)

# POKOJ 3'
M=int(2*K/5)
N=int(2*K/5)
D=[int(M*N/4+k*M) for k in range(0,int(N/2))]
Wi=range(int(M/4), int(3*M/4 - 1))
G1=range(int(M/4+M), int(3*M/4 - 1+M))
G2=[int((N/4+1)*M)-2+k*M for k in range(1, int(N/2))]
Tops=(int(3/5*K), int(2/5*K*(K-1)+3/5*K-M-20),K-1, int(2/5*K**2-1))

pokoj3=Room(M,N,Wi,D,Tops,G1,G2)

mieszkanie=Apartment([pokoj1,pokoj2,pokoj3])
for t in range(1,len(times)):
  okno=float(chlodno[t])+273.15
  pokoj1.update(t, okno,2)
  pokoj2.update(t, okno,2)
  pokoj3.update(t, okno,2)
  mieszkanie.merge(t)


  # Tworzenie animacji
X,Y=np.meshgrid(np.linspace(0,50,K), np.linspace(0,50,K))
fig, ax = plt.subplots(figsize=(4, 4))
pcm = ax.pcolormesh(X, Y, mieszkanie.u[0,:].reshape(K,K), shading='auto')
colorbar = fig.colorbar(pcm, ax=ax)
ax.set_title("Ewolucja mapy ciepła")
ax.set_xlabel("x")
ax.set_ylabel("y")

# Funkcja aktualizująca ramki animacji
def update(frame):
    data = mieszkanie.u[frame,:].reshape(K,K)
    pcm.set_array(data.ravel())

    # Dynamiczna aktualizacja zakresów
    pcm.set_clim(vmin=15, vmax=25)
    colorbar.update_normal(pcm)  # Odświeżenie zakresu colorbar

    ax.set_title(f"Na dworze: chłodno, t = {frame:.2f}")
    return pcm,

step = 5
frames = range(0, len(times), step)
ani = animation.FuncAnimation(fig, update, frames=frames, blit=False)
plt.close()
plt.rc('animation', html='jshtml')
ani
ani.save("chlodno_sciany.gif", writer="pillow")
from google.colab import files
files.download("chlodno_sciany.gif")

[2101 2198 2201 2298 2301 2398 2401 2498 2501 2598 2601 2698 2701 2798
 2801 2898 2901 2998 3001 3098 3101 3198 3201 3298 3301 3398 3401 3498
 3501 3598 3601 3698 3701 3798 3801 3898 3901 3998]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
okno=float(zimno[0])+273.15
#POKOJ 1
M=K
N=int(3*K/5)
D=range(int(M/5), int(2*M/5))
Wi1=range(int(N*M - 1-M/10-M/5), int(N*M - 1-M/10))
Wi2=range(int(N*M-1-4*M/5), int(N*M-1-3*M/5))
Wi = np.unique(np.concatenate([Wi1,Wi2]))
G11=range(int(N*M-1-13*M/10), int(N*M-1-11*M/10))
G12=range(int(N*M-1-9*M/5), int(N*M-1-8*M/5))
G1 = np.unique(np.concatenate([G11,G12]))
G21=[int(N*M/3+k*M+1) for k in range (1,int(N/3))]
G22=[int(N*M/3+k*M+1+(M-1)-2) for k in range (1,int(N/3))]
G2 = np.unique(np.concatenate([G21,G22]))
Tops=(int(2*K**2/5), int(K**2-K), int(2*K**2/5+K-1), int(K**2-1)) #LB,LU,RB,RU


pokoj1=Room(M,N,Wi,D,Tops,G1,G2)

#POKOJ 2
M=int(3*K/5)
N=int(2*K/5)
D1=range(int(N*M-1-2*M/3), int(N*M-1-M/3))
D2=[int(M*N/4+k*M-1) for k in range(1,int(N/2)+1)]
D = [D1,D2]
Wi=range(int(M/3), int(2*M/3 - 1))
G1=range(int(4*M/3), int(5*M/3-1))
G2=[int(N/4*M+k*M+1) for k in range(1, int(N/2))]
Tops=(0, int(2/5*K*(K-1)-M), int(3/5*K-1), int(2/5*K*(K-1)+3/5*K-1-M))

pokoj2=Room(M,N,Wi,D,Tops,G1,G2)

# POKOJ 3'
M=int(2*K/5)
N=int(2*K/5)
D=[int(M*N/4+k*M) for k in range(0,int(N/2))]
Wi=range(int(M/4), int(3*M/4 - 1))
G1=range(int(M/4+M), int(3*M/4 - 1+M))
G2=[int((N/4+1)*M)-2+k*M for k in range(1, int(N/2))]
Tops=(int(3/5*K), int(2/5*K*(K-1)+3/5*K-M-20),K-1, int(2/5*K**2-1))

pokoj3=Room(M,N,Wi,D,Tops,G1,G2)

mieszkanie=Apartment([pokoj1,pokoj2,pokoj3])
for t in range(1,len(times)):
  okno=float(zimno[t])+273.15
  pokoj1.update(t, okno,2)
  pokoj2.update(t, okno,2)
  pokoj3.update(t, okno,2)
  mieszkanie.merge(t)

  # Tworzenie animacji
X,Y=np.meshgrid(np.linspace(0,50,K), np.linspace(0,50,K))
fig, ax = plt.subplots(figsize=(4, 4))
pcm = ax.pcolormesh(X, Y, mieszkanie.u[0,:].reshape(K,K), shading='auto')
colorbar = fig.colorbar(pcm, ax=ax)
ax.set_title("Ewolucja mapy ciepła")
ax.set_xlabel("x")
ax.set_ylabel("y")

# Funkcja aktualizująca ramki animacji
def update(frame):
    data = mieszkanie.u[frame,:].reshape(K,K)
    pcm.set_array(data.ravel())

    # Dynamiczna aktualizacja zakresów
    pcm.set_clim(vmin=15, vmax=25)
    colorbar.update_normal(pcm)  # Odświeżenie zakresu colorbar

    ax.set_title(f"Na dworze: zimno, t = {frame:.2f}")
    return pcm,

step = 5
frames = range(0, len(times), step)
ani = animation.FuncAnimation(fig, update, frames=frames, blit=False)
plt.close()
plt.rc('animation', html='jshtml')
ani
ani.save("zimno_sciany.gif", writer="pillow")
from google.colab import files
files.download("zimno_sciany.gif")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
okno=float(bardzo_zimno[0])+273.15
#POKOJ 1
M=K
N=int(3*K/5)
D=range(int(M/5), int(2*M/5))
Wi1=range(int(N*M - 1-M/10-M/5), int(N*M - 1-M/10))
Wi2=range(int(N*M-1-4*M/5), int(N*M-1-3*M/5))
Wi = np.unique(np.concatenate([Wi1,Wi2]))
G11=range(int(N*M-1-13*M/10), int(N*M-1-11*M/10))
G12=range(int(N*M-1-9*M/5), int(N*M-1-8*M/5))
G1 = np.unique(np.concatenate([G11,G12]))
G21=[int(N*M/3+k*M+1) for k in range (1,int(N/3))]
G22=[int(N*M/3+k*M+1+(M-1)-2) for k in range (1,int(N/3))]
G2 = np.unique(np.concatenate([G21,G22]))
print(G2)
Tops=(int(2*K**2/5), int(K**2-K), int(2*K**2/5+K-1), int(K**2-1)) #LB,LU,RB,RU


pokoj1=Room(M,N,Wi,D,Tops,G1,G2)

#POKOJ 2
M=int(3*K/5)
N=int(2*K/5)
D1=range(int(N*M-1-2*M/3), int(N*M-1-M/3))
D2=[int(M*N/4+k*M-1) for k in range(1,int(N/2)+1)]
D = [D1,D2]
Wi=range(int(M/3), int(2*M/3 - 1))
G1=range(int(4*M/3), int(5*M/3-1))
G2=[int(N/4*M+k*M+1) for k in range(1, int(N/2))]
Tops=(0, int(2/5*K*(K-1)-M), int(3/5*K-1), int(2/5*K*(K-1)+3/5*K-1-M))

pokoj2=Room(M,N,Wi,D,Tops,G1,G2)

# POKOJ 3'
M=int(2*K/5)
N=int(2*K/5)
D=[int(M*N/4+k*M) for k in range(0,int(N/2))]
Wi=range(int(M/4), int(3*M/4 - 1))
G1=range(int(M/4+M), int(3*M/4 - 1+M))
G2=[int((N/4+1)*M)-2+k*M for k in range(1, int(N/2))]
Tops=(int(3/5*K), int(2/5*K*(K-1)+3/5*K-M-20),K-1, int(2/5*K**2-1))

pokoj3=Room(M,N,Wi,D,Tops,G1,G2)

mieszkanie=Apartment([pokoj1,pokoj2,pokoj3])

for t in range(1,len(times)):
  okno=float(bardzo_zimno[t])+273.15
  pokoj1.update(t, okno,2)
  pokoj2.update(t, okno,2)
  pokoj3.update(t, okno,2)
  mieszkanie.merge(t)

  # Tworzenie animacji
X,Y=np.meshgrid(np.linspace(0,50,K), np.linspace(0,50,K))
fig, ax = plt.subplots(figsize=(4, 4))
pcm = ax.pcolormesh(X, Y, mieszkanie.u[0,:].reshape(K,K), shading='auto')
colorbar = fig.colorbar(pcm, ax=ax)
ax.set_title("Ewolucja mapy ciepła")
ax.set_xlabel("x")
ax.set_ylabel("y")

# Funkcja aktualizująca ramki animacji
def update(frame):
    data = mieszkanie.u[frame,:].reshape(K,K)
    pcm.set_array(data.ravel())

    # Dynamiczna aktualizacja zakresów
    pcm.set_clim(vmin=15, vmax=25)
    colorbar.update_normal(pcm)  # Odświeżenie zakresu colorbar

    ax.set_title(f"Na dworze: bardzo zimno, t = {frame:.2f}")
    return pcm,

step = 5
frames = range(0, len(times), step)
ani = animation.FuncAnimation(fig, update, frames=frames, blit=False)
plt.close()
plt.rc('animation', html='jshtml')
ani
ani.save("bardzo_zimno_sciany.gif", writer="pillow")
from google.colab import files
files.download("bardzo_zimno_sciany.gif")

[2101 2198 2201 2298 2301 2398 2401 2498 2501 2598 2601 2698 2701 2798
 2801 2898 2901 2998 3001 3098 3101 3198 3201 3298 3301 3398 3401 3498
 3501 3598 3601 3698 3701 3798 3801 3898 3901 3998]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Czy wyłączać grzejniki?
Przeanalizujmy teraz, o ile zmieni się nasz wydatek na ciepło kiedy wyłączymy grzejniki w godzinach 10:00-16:00

In [41]:
import numpy as np
import matplotlib.pyplot as plt

class Room:
    def __init__(self, m,n, windows, doors, tops, heater1, heater2):
        self.M=m
        self.N=n
        self.windows=windows
        self.doors=doors
        self.tops=tops
        self.h1=heater1
        self.h2=heater2

        ib1=[i for i in range(m*n) if i%m==0 ]
        ib2=[i for i in range(m*n) if (i+1)%m==0]
        ib3=[i for i in range(m*n) if i<m]
        ib4=[i for i in range(m*n) if i>m*(n-1)]

        ib1prim=[i for i in range(m*n) if i%m==1 ]
        ib2prim=[i for i in range(m*n) if (i+2)%m==0]
        ib3prim=[i+m for i in range(m*n) if i<m]
        ib4prim=[i-m for i in range(m*n) if i>m*(n-1)]

        self.walls=ib1+ib2+ib3+ib4
        self.wallsprim=ib1prim+ib2prim+ib3prim+ib4prim

        self.laplasjan()
        self.x=np.linspace(0,self.M,1)
        self.y=np.linspace(0,self.N,1)
        self.X,self.Y=np.meshgrid(self.x,self.y)
        self.u=np.zeros((len(times), self.M * self.N))
        self.u[0, :] = [18+273.15]*(self.N*self.M)
        self.u[0,self.windows]=[okno]*len(self.windows)

        self.epsilon=800/(1.2*1*0.05*1005)
        self.power=0

    def laplasjan(self):
        D2x = np.diag(-2 * np.ones(self.M)) + np.diag(np.ones(self.M- 1), 1) + np.diag(np.ones(self.M - 1), -1)
        D2y = np.diag(-2 * np.ones(self.N)) + np.diag(np.ones(self.N - 1), 1) + np.diag(np.ones(self.N - 1), -1)
        self.L = np.kron(np.eye(self.N), D2x) + np.kron(D2y, np.eye(self.M))
    def update(self,t, okno, typ=1):
        self.u[t, :] = self.u[t-1, :] + (ht / hx**2) * np.matmul(self.L, self.u[t-1, :])
        if(self.u[t, int(self.M*self.N/2+ self.M/2)]>21+273.15 or t in range(200,320)): heater_on=0
        else: heater_on=1
        if(typ==1):
          self.u[t, self.h1] += self.epsilon * heater_on
          self.power+=self.epsilon * heater_on
        if(typ==2):
          self.u[t, self.h2] += self.epsilon * heater_on
          self.power+=self.epsilon * heater_on
        #Warunki brzegowe
        #temp=self.u[t, self.doors]
        #print(temp)
        self.u[t, self.walls] = self.u[t, self.wallsprim]
        self.u[t, self.windows] = [okno]*len(self.windows)
        #self.u[t, self.doors] = temp




class Apartment:
    def __init__(self, rooms):
        self.rooms = rooms

        ib1=[i for i in range(K*K) if i%K==0 ]
        ib2=[i for i in range(K*K) if (i+1)%K==0]
        ib3=[i for i in range(K*K) if i<K]
        ib4=[i for i in range(K*K) if i>=K*(K-1)]
        ib5=[i for i in range(int(2/5*K*K), int(2/5*K*K+K)) ]
        ib6=[i for i in range(0, int(2/5*K*K)) if i%K==int(3/5*K)]
        ib7=[i for i in range(int(2/5*K*K-K), int(2/5*K*K)) ]
        ib8=[i for i in range(0, int(2/5*K*K)) if (i+1)%K==int(3/5*K)]

        i=len(ib8)
        del ib8[int(i*1/4): int(i*3/4)]
        del ib6[int(i*1/4): int(i*3/4)]
        del ib5[20:40]
        del ib7[20:40]

        del ib3[20:39]
        del ib3[51:69]
        del ib4[19:39]
        del ib4[49:69]

        self.walls=ib1+ib2+ib3+ib4+ib5+ib6+ib7+ib8

        self.u=np.zeros((len(times), K*K))
        self.merge(0)

    def update(self, t, typ):
        for room in self.rooms:
            room.update(t, typ)
    def merge(self,t):
     # 1 3
     # 0 2
      pok1=self.rooms[0]
      pok2=self.rooms[1]
      pok3=self.rooms[2]

      pok1.u[t, pok1.doors], pok2.u[t, pok2.doors[0]] = (pok1.u[t, pok1.doors]+ pok2.u[t, pok2.doors[0]])/2, (pok1.u[t, pok1.doors]+ pok2.u[t, pok2.doors[0]])/2
      pok2.u[t, pok2.doors[1]], pok3.u[t, pok3.doors] = (pok2.u[t, pok2.doors[1]]+ pok3.u[t, pok3.doors])/2, (pok2.u[t, pok2.doors[1]]+ pok3.u[t, pok3.doors])/2


      self.u[t, range(pok1.tops[0], pok1.tops[3]+1)]=pok1.u[t,:]-[273.15]*6000
      p2=[i for i in range(pok2.tops[0], pok2.tops[3]+1) if i % K <= pok2.tops[2]]
      p3=[i for i in range(pok3.tops[0], pok3.tops[3]+1) if i % K >= pok3.tops[0]]
      self.u[t, p2]=pok2.u[t,:]-[273.15]*2400
      self.u[t, p3]=pok3.u[t,:]-[273.15]*1600
      self.u[t,self.walls]=[np.nan]*len(self.walls)



In [42]:
#POKOJ 1
M=K
N=int(3*K/5)
D=range(int(M/5), int(2*M/5))
Wi1=range(int(N*M - 1-M/10-M/5), int(N*M - 1-M/10))
Wi2=range(int(N*M-1-4*M/5), int(N*M-1-3*M/5))
Wi = np.unique(np.concatenate([Wi1,Wi2]))
G11=range(int(N*M-1-13*M/10), int(N*M-1-11*M/10))
G12=range(int(N*M-1-9*M/5), int(N*M-1-8*M/5))
G1 = np.unique(np.concatenate([G11,G12]))
G21=[int(N*M/3+k*M+1) for k in range (1,int(N/3))]
G22=[int(M*(N/3+1)-2+k*M) for k in range (int(N/3)+1,int(2*N/3))]
G2 = np.unique(np.concatenate([G21,G22]))
Tops=(int(2*K**2/5), int(K**2-K), int(2*K**2/5+K-1), int(K**2-1)) #LB,LU,RB,RU


pokoj1=Room(M,N,Wi,D,Tops,G1,G2)

#POKOJ 2
M=int(3*K/5)
N=int(2*K/5)
D1=range(int(N*M-1-2*M/3), int(N*M-1-M/3))
D2=[int(M*N/4+k*M-1) for k in range(1,int(N/2)+1)]
D = [D1,D2]
Wi=range(int(M/3), int(2*M/3 - 1))
G1=range(int(4*M/3), int(5*M/3-1))
G2=[int(N/4*M+k*M+1) for k in range(1, int(N/2))]
Tops=(0, int(2/5*K*(K-1)-M), int(3/5*K-1), int(2/5*K*(K-1)+3/5*K-1-M))

pokoj2=Room(M,N,Wi,D,Tops,G1,G2)

# POKOJ 3'
M=int(2*K/5)
N=int(2*K/5)
D=[int(M*N/4+k*M) for k in range(0,int(N/2))]
Wi=range(int(M/4), int(3*M/4 - 1))
G1=range(int(M/4+M), int(3*M/4 - 1+M))
G2=[int((N/4+1)*M)-2+k*M for k in range(1, int(N/2))]
Tops=(int(3/5*K), int(2/5*K*(K-1)+3/5*K-M-20),K-1, int(2/5*K**2-1))

pokoj3=Room(M,N,Wi,D,Tops,G1,G2)

mieszkanie=Apartment([pokoj1,pokoj2,pokoj3])
for t in range(1,len(times)):
  okno=float(chlodno[t])+273.15
  pokoj1.update(t, okno,1)
  pokoj2.update(t, okno,1)
  pokoj3.update(t, okno,1)
  mieszkanie.merge(t)
cieplo=pokoj1.power+pokoj2.power+pokoj3.power
powersprim.append(cieplo)

#POKOJ 1
M=K
N=int(3*K/5)
D=range(int(M/5), int(2*M/5))
Wi1=range(int(N*M - 1-M/10-M/5), int(N*M - 1-M/10))
Wi2=range(int(N*M-1-4*M/5), int(N*M-1-3*M/5))
Wi = np.unique(np.concatenate([Wi1,Wi2]))
G11=range(int(N*M-1-13*M/10), int(N*M-1-11*M/10))
G12=range(int(N*M-1-9*M/5), int(N*M-1-8*M/5))
G1 = np.unique(np.concatenate([G11,G12]))
G21=[int(N*M/3+k*M+1) for k in range (1,int(N/3))]
G22=[int(M*(N/3+1)-2+k*M) for k in range (int(N/3)+1,int(2*N/3))]
G2 = np.unique(np.concatenate([G21,G22]))
Tops=(int(2*K**2/5), int(K**2-K), int(2*K**2/5+K-1), int(K**2-1)) #LB,LU,RB,RU


pokoj1=Room(M,N,Wi,D,Tops,G1,G2)

#POKOJ 2
M=int(3*K/5)
N=int(2*K/5)
D1=range(int(N*M-1-2*M/3), int(N*M-1-M/3))
D2=[int(M*N/4+k*M-1) for k in range(1,int(N/2)+1)]
D = [D1,D2]
Wi=range(int(M/3), int(2*M/3 - 1))
G1=range(int(4*M/3), int(5*M/3-1))
G2=[int(N/4*M+k*M+1) for k in range(1, int(N/2))]
Tops=(0, int(2/5*K*(K-1)-M), int(3/5*K-1), int(2/5*K*(K-1)+3/5*K-1-M))

pokoj2=Room(M,N,Wi,D,Tops,G1,G2)

# POKOJ 3'
M=int(2*K/5)
N=int(2*K/5)
D=[int(M*N/4+k*M) for k in range(0,int(N/2))]
Wi=range(int(M/4), int(3*M/4 - 1))
G1=range(int(M/4+M), int(3*M/4 - 1+M))
G2=[int((N/4+1)*M)-2+k*M for k in range(1, int(N/2))]
Tops=(int(3/5*K), int(2/5*K*(K-1)+3/5*K-M-20),K-1, int(2/5*K**2-1))

pokoj3=Room(M,N,Wi,D,Tops,G1,G2)

mieszkanie=Apartment([pokoj1,pokoj2,pokoj3])
for t in range(1,len(times)):
  okno=float(zimno[t])+273.15
  pokoj1.update(t, okno,1)
  pokoj2.update(t, okno,1)
  pokoj3.update(t, okno,1)
  mieszkanie.merge(t)
cieplo=pokoj1.power+pokoj2.power+pokoj3.power
powersprim.append(cieplo)

#POKOJ 1
M=K
N=int(3*K/5)
D=range(int(M/5), int(2*M/5))
Wi1=range(int(N*M - 1-M/10-M/5), int(N*M - 1-M/10))
Wi2=range(int(N*M-1-4*M/5), int(N*M-1-3*M/5))
Wi = np.unique(np.concatenate([Wi1,Wi2]))
G11=range(int(N*M-1-13*M/10), int(N*M-1-11*M/10))
G12=range(int(N*M-1-9*M/5), int(N*M-1-8*M/5))
G1 = np.unique(np.concatenate([G11,G12]))
G21=[int(N*M/3+k*M+1) for k in range (1,int(N/3))]
G22=[int(M*(N/3+1)-2+k*M) for k in range (int(N/3)+1,int(2*N/3))]
G2 = np.unique(np.concatenate([G21,G22]))
Tops=(int(2*K**2/5), int(K**2-K), int(2*K**2/5+K-1), int(K**2-1)) #LB,LU,RB,RU


pokoj1=Room(M,N,Wi,D,Tops,G1,G2)

#POKOJ 2
M=int(3*K/5)
N=int(2*K/5)
D1=range(int(N*M-1-2*M/3), int(N*M-1-M/3))
D2=[int(M*N/4+k*M-1) for k in range(1,int(N/2)+1)]
D = [D1,D2]
Wi=range(int(M/3), int(2*M/3 - 1))
G1=range(int(4*M/3), int(5*M/3-1))
G2=[int(N/4*M+k*M+1) for k in range(1, int(N/2))]
Tops=(0, int(2/5*K*(K-1)-M), int(3/5*K-1), int(2/5*K*(K-1)+3/5*K-1-M))

pokoj2=Room(M,N,Wi,D,Tops,G1,G2)

# POKOJ 3'
M=int(2*K/5)
N=int(2*K/5)
D=[int(M*N/4+k*M) for k in range(0,int(N/2))]
Wi=range(int(M/4), int(3*M/4 - 1))
G1=range(int(M/4+M), int(3*M/4 - 1+M))
G2=[int((N/4+1)*M)-2+k*M for k in range(1, int(N/2))]
Tops=(int(3/5*K), int(2/5*K*(K-1)+3/5*K-M-20),K-1, int(2/5*K**2-1))

pokoj3=Room(M,N,Wi,D,Tops,G1,G2)

mieszkanie=Apartment([pokoj1,pokoj2,pokoj3])
for t in range(1,len(times)):
  okno=float(bardzo_zimno[t])+273.15
  pokoj1.update(t, okno,1)
  pokoj2.update(t, okno,1)
  pokoj3.update(t, okno,1)
  mieszkanie.merge(t)
cieplo=pokoj1.power+pokoj2.power+pokoj3.power
powersprim.append(cieplo)



In [29]:
print(powers)
print(powersprim)

[14182.421227197432]
[10029.850746268694, 11820.895522388117, 14248.75621890556]
